In [1]:
import pandas as pd
import os

#get data
DIR = os.getcwd() + "/nfl_predictor"
df = pd.read_csv(DIR + "/nfl_games.csv", index_col=0)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
# sort data
df = df.sort_values("date")
df = df.reset_index(drop=True)
del df["index_opp"]

# add target column
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)
df["target"][pd.isnull(df["target"])] = 2
# converts target boolean value to 1 or 0
df["target"] = df["target"].astype(int,errors="ignore")

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_49421/3382075958.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [3]:
# picking columns based on feature selector
selected_columns = ['cmp',
 'att',
 'rate',
 'tgt',
 'rec',
 'lng.2',
 'att_max',
 'int_max',
 'att.1_max',
 'yds.2_max',
 'td.2_max',
 'fl_max',
 'total',
 'int_opp',
 'sk_opp',
 'lng_opp',
 'att.1_opp',
 'yds.2_opp',
 'lng.1_opp',
 'fmb_opp',
 'fl_opp',
 'int_max_opp',
 'lng_max_opp',
 'lng.1_max_opp',
 'rec_max_opp',
 'yds.3_max_opp',
 'td.2_max_opp',
 'lng.2_max_opp',
 'fl_max_opp',
 'total_opp',
 'target']

removed_columns = df.columns[~df.columns.isin(selected_columns)]
df = df.drop(columns=removed_columns)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4380 entries, 0 to 4379
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cmp            4380 non-null   float64
 1   att            4380 non-null   float64
 2   rate           4380 non-null   float64
 3   tgt            4380 non-null   float64
 4   rec            4380 non-null   float64
 5   lng.2          4380 non-null   float64
 6   att_max        4380 non-null   float64
 7   int_max        4380 non-null   float64
 8   att.1_max      4380 non-null   float64
 9   yds.2_max      4380 non-null   float64
 10  td.2_max       4380 non-null   float64
 11  fl_max         4380 non-null   float64
 12  total          4380 non-null   int64  
 13  int_opp        4380 non-null   float64
 14  sk_opp         4380 non-null   float64
 15  lng_opp        4380 non-null   float64
 16  att.1_opp      4380 non-null   float64
 17  yds.2_opp      4380 non-null   float64
 18  lng.1_opp    

In [5]:
# time to split data
from sklearn.model_selection import train_test_split

X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=8)

ModuleNotFoundError: No module named 'sklearn.model_selection'

In [13]:
# now we set up a pipeline
# sequentially applies a list of transforms and a final estimator
# purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different params
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier

estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBClassifier(random_state = 8)) # can customize objective function with the objective parameter 
]

pipe = Pipeline(steps=estimators)
pipe

/Users/brandonhart/Documents/nfl-predict/.conda/lib/python3.11/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The TargetEncoder or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
/Users/brandonhart/Documents/nfl-predict/.conda/lib/python3.11/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMi

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

Pipeline(steps=[('encoder', TargetEncoder()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
   

In [7]:
%conda update scikit-learn

Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/brandonhart/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/11" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-arm64/current_repodata.json HTTP/11" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/11" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-arm64/current_repodata.json HTTP/11" 304 0
done
Solving environment: done


==> WARNING: A newer vers